Algoritmo de Shor
========================
    
El algoritmo de Shor consiste en reducir el problema de factorizar un número *N* al problema de encontrar el orden *r* de un entero *a* respecto a *N*, éste paso sólo se puede hacer con recursos superpolinomiales en la computación clásica con los algoritmos conocidos, pero hay un algoritmo de complejidad polinomial usando una computadora cuántica. 
    
El proceso general es el siguiente:

1. Se obtiene un número aleatorio *a* que cumpla con $a<N$ y $mcd(a,N)=1$, si $mcd(a,N)\ne 1$ entonces a es un divisor de *N* y ya tenemos un factor de los que estamos buscando.

1. Se calcula el orden de *a* respecto a *N*, es decir el número que cumpla $r = min(\{x \in Z | a^x mod N = 1\})$.

1. Si *r* es un número impar se regresa al paso 1, de lo contrario se calcula $d=mcd(a^{r/2}-1,N)$, si $d\ne 1$ entonces *d* es un factor de  N.

Paso 2 y su implementación cuántica
---------------------------------------

El paso de encontrar el orden de *a* respecto a *N* es polinomial en una computadora cuántica de acuerdo al siguiente algoritmo:

1. Crear el siguiente circuito, con el primer y segundo registros de un tamaño *n* que cumpla $N^2 \le 2^n \le 2N^2$, El primer registro inicializado en $|0\rangle ^{\otimes n}$ y el segundo en $|0\rangle ^{\otimes n-1}\otimes|1\rangle$
![circuito](circuito.png "circuito")
1. La salida de la primera compuerta $QFT_{2^n}$ (el punto marcado por **1** en el circuito) es el estado $2^{-n/2}(|0\rangle+|1\rangle)^{\otimes n}\otimes|0\rangle^{\otimes n-1}|1\rangle$.
1. La compuerta marcada por $U_a^x$ hace la transformación $|y\rangle \rightarrow |a^xmodN\rangle$ donde $x$ es el registro de control descodificándolo en binario ($|5\rangle = |1\rangle|0\rangle|1\rangle$), usando:
$$ \frac{1}{\sqrt{2^n}}(|0\rangle + |1\rangle)^{\otimes n}= \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2n-1}|x\rangle $$
$$\Rightarrow  \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2n-1} U_a^x |x\rangle |1\rangle =  \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2n-1}|x\rangle |a^x modN\rangle = \frac{1}{\sqrt{2^n}}\sum_{b=0}^{r-1}\sum_{z=0}^{m_b-1}|zr+b\rangle |a^b modN\rangle $$
con $m_b$ de tal forma que $zr+b \le 2^n-1$ es un valor que depende del actual $b$ , éste estado es el marcado en el diagrama por **2**
1. Si se mide el segundo registro el estado colapsa a algún valor de $b$ y el primer registro queda en el estado periódico, y el periodo del estado es el orden de $a$ respecto a $N$
$$\frac{1}{\sqrt{m_b}}\sum_{z=0}^{m_b -1} |zr+b\rangle$$
Aplicando la compuerta $QFT_{2^n}^{-1}$ al primer registro la salida (**punto 3**) es una superposición de estados $|x\rangle$ de forma que $x/2^n \approx k/r$ para algún valor de $k<r$
1. Usando el algoritmo de fracciones contínuas se desarrolla la fracción $x/2^n$ medida hasta encontrar un convergente $\frac{c_1}{r_1}$que cumpla $$\|\frac{x}{2^n}-\frac{c_1}{r_1}\| \le \frac{1}{2^{\frac{n-1}{2}}}$$
La medición de $x/2^n$ tiene probabilidad $\frac{4}{r\pi^2}$ de arrojar una fracción útil para este paso, hay que repetir desde el paso 1 hasta obtener dos convergentes distintos $\frac{c_1}{r_1}$, $\frac{c_2}{r_2}$.
1. Después se calcula $r = mcm(r_1,r_2)$, si $a^r modN = 1$ entonces $r$ es el orden de $a$ respecto a $N$, de lo contrario se falló y hay que intentar con un nuevo valor de $a$.

In [166]:
N = 15
a=7
n = int(ceil(2*log2(N)))
baseQ=big(int(big(2)^(2*n)))
Ψ0 = numtoQvector(xytoQstateNum(0,1,n),n)
ρ0 = Ψ0*Ψ0';
pflip = 0.0001

0.0001

In [198]:
ρ1=aplicaError(ρ0,pflip,flips)

4096x4096 sparse matrix with 13 Float64 entries:
	[1   ,    1]  =  8.33333e-6
	[65  ,   65]  =  0.9999
	[66  ,   66]  =  8.33333e-6
	[67  ,   67]  =  8.33333e-6
	[69  ,   69]  =  8.33333e-6
	[73  ,   73]  =  8.33333e-6
	[81  ,   81]  =  8.33333e-6
	[97  ,   97]  =  8.33333e-6
	[193 ,  193]  =  8.33333e-6
	[321 ,  321]  =  8.33333e-6
	[577 ,  577]  =  8.33333e-6
	[1089, 1089]  =  8.33333e-6
	[2113, 2113]  =  8.33333e-6

In [199]:
ρ2=QFT*ρ1*QFT'
ρ3=aplicaError(ρ2,pflip,flips)

4096x4096 sparse matrix with 90112 Complex{Float64} entries:
	[1   ,    1]  =  2.60404e-7+0.0im
	[2   ,    1]  =  2.604e-7+2.58561e-12im
	[3   ,    1]  =  2.604e-7+2.47925e-12im
	[4   ,    1]  =  2.60398e-7+1.60248e-12im
	[5   ,    1]  =  2.604e-7+2.26757e-12im
	[6   ,    1]  =  2.60396e-7+2.73398e-12im
	[7   ,    1]  =  2.60398e-7+1.2875e-12im
	[8   ,    1]  =  2.60397e-7+3.1549e-13im
	[9   ,    1]  =  2.604e-7+1.85233e-12im
	[10  ,    1]  =  2.60396e-7+3.34008e-12im
	⋮
	[3190, 3200]  =  2.17014e-12+0.0im
	[3191, 3200]  =  2.17014e-12+0.0im
	[3192, 3200]  =  2.17014e-12+0.0im
	[3193, 3200]  =  2.17014e-12+0.0im
	[3194, 3200]  =  2.17014e-12+0.0im
	[3195, 3200]  =  2.17014e-12+0.0im
	[3196, 3200]  =  2.17014e-12+0.0im
	[3197, 3200]  =  2.17014e-12+0.0im
	[3198, 3200]  =  2.17014e-12+0.0im
	[3199, 3200]  =  2.17014e-12+0.0im
	[3200, 3200]  =  2.17014e-12+0.0im

In [200]:
ρ4 = operadorMod*ρ3*operadorMod'
ρ5 = aplicaError(ρ4,pflip,flips)

4096x4096 sparse matrix with 221184 Complex{Float64} entries:
	[1   ,    1]  =  3.90586e-7+2.75727e-33im
	[2   ,    1]  =  2.60387e-7+2.58544e-12im
	[3   ,    1]  =  3.90579e-7+4.95818e-12im
	[4   ,    1]  =  2.60385e-7+1.60237e-12im
	[5   ,    1]  =  3.90579e-7+4.53483e-12im
	[6   ,    1]  =  2.60383e-7+2.7338e-12im
	[7   ,    1]  =  3.90574e-7+2.57482e-12im
	[8   ,    1]  =  2.60384e-7+3.15481e-13im
	[9   ,    1]  =  3.90579e-7+3.70441e-12im
	[10  ,    1]  =  2.60383e-7+3.33986e-12im
	⋮
	[3702, 3712]  =  5.42535e-17+0.0im
	[3703, 3712]  =  5.42535e-17+0.0im
	[3704, 3712]  =  5.42535e-17+0.0im
	[3705, 3712]  =  5.42535e-17+0.0im
	[3706, 3712]  =  5.42535e-17+0.0im
	[3707, 3712]  =  5.42535e-17+0.0im
	[3708, 3712]  =  5.42535e-17+0.0im
	[3709, 3712]  =  5.42535e-17+0.0im
	[3710, 3712]  =  5.42535e-17+0.0im
	[3711, 3712]  =  5.42535e-17+0.0im
	[3712, 3712]  =  5.42535e-17+0.0im

In [201]:
ρ6 = QFTinv*ρ5*QFTinv'
ρf=aplicaError(ρ6,pflip,flips)

4096x4096 sparse matrix with 495616 Complex{Float64} entries:
	[1   ,    1]  =  2.08288e-5+1.91576e-27im
	[2   ,    1]  =  -1.00064e-21+3.84849e-24im
	[3   ,    1]  =  -4.15091e-22-1.82142e-23im
	[4   ,    1]  =  -1.82412e-21+1.9501e-21im
	[5   ,    1]  =  -8.96766e-22-3.11794e-22im
	[6   ,    1]  =  -1.14251e-21-2.89103e-21im
	[7   ,    1]  =  -9.72172e-21-1.29419e-21im
	[8   ,    1]  =  7.14754e-21-1.00154e-22im
	[9   ,    1]  =  -1.35059e-21-2.95426e-23im
	[10  ,    1]  =  2.32039e-21-1.51884e-21im
	⋮
	[3958, 3968]  =  -3.87045e-50-2.5727e-49im
	[3959, 3968]  =  -2.21325e-49-1.74615e-49im
	[3960, 3968]  =  -2.50336e-49-6.34963e-50im
	[3961, 3968]  =  4.13091e-49+2.64568e-51im
	[3962, 3968]  =  3.50325e-50-5.00672e-49im
	[3963, 3968]  =  7.66335e-50-1.01959e-48im
	[3964, 3968]  =  3.20219e-50+3.33173e-49im
	[3965, 3968]  =  2.05451e-49-2.61831e-49im
	[3966, 3968]  =  4.11631e-49+2.98141e-49im
	[3967, 3968]  =  4.32432e-50-1.21428e-49im
	[3968, 3968]  =  6.69266e-49-2.15304e-50im

In [192]:
Ψf=QFTinv*operadorMod*QFT*Ψ0

4096x1 sparse matrix with 128 Complex{Float64} entries:
	[65  ,    1]  =  0.5+0.0im
	[66  ,    1]  =  -3.64292e-17-3.51282e-17im
	[67  ,    1]  =  -5.20417e-18+0.0im
	[68  ,    1]  =  -9.71445e-17+6.93889e-17im
	[69  ,    1]  =  -1.04083e-16+6.93889e-18im
	[70  ,    1]  =  -5.20417e-17-1.02349e-16im
	[71  ,    1]  =  -2.17708e-16-6.59195e-17im
	[72  ,    1]  =  1.50054e-16+1.40513e-16im
	[73  ,    1]  =  -3.24562e-16-7.28584e-17im
	[74  ,    1]  =  1.20997e-16-1.26635e-16im
	⋮
	[374 ,    1]  =  2.01228e-16+8.32667e-17im
	[375 ,    1]  =  1.05818e-16-3.05311e-16im
	[376 ,    1]  =  -1.04777e-15+2.03483e-15im
	[377 ,    1]  =  -2.0782e-15+3.50414e-16im
	[378 ,    1]  =  3.46945e-17-1.48666e-15im
	[379 ,    1]  =  5.82867e-16-2.24647e-15im
	[380 ,    1]  =  -9.36751e-17+1.01308e-15im
	[381 ,    1]  =  3.92048e-16-1.68875e-15im
	[382 ,    1]  =  1.90646e-15+5.82867e-16im
	[383 ,    1]  =  -8.67362e-18+2.38091e-16im
	[384 ,    1]  =  1.02002e-15-4.94396e-17im

In [202]:
probfinal = Ψf'*ρf*Ψf

1x1 sparse matrix with 1 Complex{Float64} entries:
	[1, 1]  =  0.999692+1.37325e-23im